In [4]:
import numpy as np
import cv2 as cv
import os

In [2]:
def load_images(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images

In [36]:
def sample_pixels(images, num_samples):
    
    # Choose pixels with largest standard deviation
    stds = np.std(images, axis=0)
    selected_pixel_value = np.argsort(stds.ravel())[::-1][:num_samples]
    sampled_pixels = np.unravel_index(selected_pixel_value, stds.shape) 
    # returns (i of chosen pixels, j of chosen pixels)
    
    # Compute the sample values for each pixel
    samples = np.zeros((num_samples, len(images)), dtype=np.float32)
    for i in range(num_samples):
        x, y = sampled_pixels[0][i], sampled_pixels[1][i]
        for j in range(len(images)):
            samples[i, j] = images[j][x, y]
    
    return samples
    # returns [ [first chosen pixel in every image], [], ..... [] ]

In [38]:
def solve_respond_curve(z,lgT,lam,w):
    #z : ith pixel of those chosen in jth image
    n = len(z) #n : we choose n pixels total for each images
    p = len(z[0])#p : we choose p images total 
    row = n*p + 1 + 254
    col = 256 + n   # row and col in matrix a
    a = [[] for i in range(row)]
    b = [[] for i in range(row)]

    #initialize matrix a(zero) and b(zero)
    for i in range(row):
        b[i].append(0)
        for j in range(col):
            a[i].append(0)

    #upper half of matrix a and b done
    for j in range(p):
        for i in range(n):
            Z = int(z[i][j])
            b[i+j*n][0] = lgT[j] * w[Z]
            for r in range(256):
                if r == Z :
                    a[i+j*n][r] = 1 * w[Z]
            a[i+j*n][i] = -1 * w[Z]

    #fix the curve by setting its middle value to 0
    a[n*p][128] = 1

    #lower half of matrix a and b done
    for k in range(n*p,n*p+254):
        for l in range(254):
            a[k+1][l] = 1 * z[l+1] * lam 
            a[k+1][l+1] = -2 * z[l+1] * lam 
            a[k+1][l+2] = 1 * z[l+1] * lam 
            
    
    A = np.array(a)
    B = np.array(b)

    print(np.shape(A))
    print(np.shape(B))
    
    #Solve the system using SVD
    x = np.linalg.lstsq(A,B)[0]
    g = x[:256]
    lE = x[256:]

    return g, lE

In [39]:
if __name__ == '__main__':
    folder = "testing_image" # for now
    images = np.array(load_images(folder))
    num_samples = 50 # for now
    
    z = sample_pixels(images[0], num_samples)
    exposure_times = [0.25, 0.5, 0.75, 1] # for now
    lambda_ = 50 # for now
    w = [value if value <= 0.5*255 else 255-value for value in range(256)]
    
    g, lnE = solve_respond_curve(z, exposure_times, lambda_, w)

IndexError: list index out of range